# Cohort Analysis
### of Resellers in a dropshipping company

The goal of this analysis is measuring the retention rate of resellers in a dropshipping company across weeks for resellers who place five orders in the first month of their activation.

The day they place their first order is day 1, the week they place their first order is week 1, and the month they place their first order is month 1.
- When place order in week 1, resellers are considered retained for weeks 1, 2, 3, 4.
- When place order in week 2, resellers are considered retained for weeks 2, 3, 4.
- When place order in week 3, resellers are considered retained for weeks 3, 4.
- When place order in week 4, resellers are considered retained for week 4.
- When place order in week 5, resellers are considered retained for weeks 5, 6, 7, 8.

And so on, and so forth.

In the end, the retion of each week is divided by the retantion of week 1 to calculate retention rate in each week.


with base1 AS (SELECT
    `orders`.`ID` AS `OrderID`,
    `orders`.`UserID` AS `UserID`,
    `orders`.`CreatedAt` AS `OrderDate`,
    `orders`.`FakeDelete` AS `FakeDelete`,
    `Users`.`TotalOrders` AS `TotalOrders`,
    `Users`.`TotalGrossOrders` AS `GrossOrders`,
    `Users`.`CreatedAt` AS `JoinDate`,
    `Vw Orders Is Net`.`isGross` AS `isGross`,
    `Vw Users First Order`.`FirstOrderAt` AS `ActivationDate`,
    -- DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) + 1 AS `Day`,
    CEILING((DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`)+1)/7) as `WeekActivation`,
    CEILING((DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`)+1)/30) as `MonthActivation`,
    ROW_NUMBER() OVER (PARTITION BY UserID ORDER BY orders.CreatedAt ASC ) AS `OrderOrder`
    FROM
    `orders`
    INNER JOIN `users` AS `Users` ON `orders`.`UserID` = `Users`.`ID`
    INNER JOIN `vw_orders_is_net` AS `Vw Orders Is Net` ON `orders`.`ID` = `Vw Orders Is Net`.`OrderID`
    INNER JOIN `vw_users_first_order` AS `Vw Users First Order` ON `Users`.`ID` = `Vw Users First Order`.`UserID`
    WHERE
    (`Vw Orders Is Net`.`isGross` = 1)
    AND (`orders`.`FakeDelete` = 0)
    ORDER BY `orders`.`UserID`),

base2 AS (SELECT UserID, MonthActivation, MAX(WeekActivation), COUNT(OrderID) AS `#Orders`  FROM base1 GROUP BY UserID, MonthActivation ORDER By UserID, MonthActivation),

fiveOrders AS (SELECT UserID, OrderDate, OrderOrder 
    FROM base1 
    WHERE UserID IN (SELECT UserID  FROM base2 WHERE MonthActivation=1 and `#Orders`>4)),
    
graduation AS (SELECT UserID, MAX(OrderDate) AS graduationDate FROM fiveOrders WHERE OrderOrder=5 GROUP BY UserID),

fiveOrdersGrad AS (SELECT fiveOrders.UserID, OrderDate, OrderOrder, graduationDate
    FROM fiveOrders JOIN graduation ON fiveOrders.UserID = graduation.UserID ),
    
fiveOrdersComplete AS (SELECT UserID,  CEILING((DATEDIFF(OrderDate, graduationDate)+1)/7) AS WeekGraduation
    FROM fiveOrdersGrad
    WHERE OrderOrder > 2
    GROUP BY WeekGraduation, UserID),
    

myWeeks AS (SELECT DISTINCT WeekGraduation AS Week FROM fiveOrdersComplete ORDER BY WeekGraduation),

myJoin AS (SELECT myWeeks.Week AS WWeek, fiveOrdersComplete.WeekGraduation as TWeek, UserID
FROM myWeeks, fiveOrdersComplete
WHERE fiveOrdersComplete.WeekGraduation BETWEEN (myWeeks.Week-(CASE WHEN MOD(myWeeks.Week, 4)> 0 THEN MOD(myWeeks.Week, 4)-1 ELSE 3 END)) AND myWeeks.Week),

myFinal AS (SELECT WWeek AS Week, COUNT(DISTINCT TWeek) AS Duration, MIN(TWeek) AS StartWeek, MAX(TWeek) AS EndWeek, COUNT(DISTINCT UserID) AS ActivesInInterval
FROM  myJoin
GROUP BY Week),

allActivated AS (SELECT COUNT(DISTINCT UserID) AS AllActivation FROM fiveOrdersComplete WHERE WeekGraduation=1)

SELECT Week, StartWeek, Duration, EndWeek, ActivesInInterval, ActivesInInterval/AllActivation AS PercentActivesInInterval
FROM  myFinal, allActivated
WHERE Week > 4;